# Data Access and Cleaning

The purpose of this notebook is to download the subsurface data from the Washington State Department of Natural Resources (DNR) [Geologic Information Portal](https://www.dnr.wa.gov/geologyportal), available for [download](https://www.dnr.wa.gov/programs-and-services/geology/publications-and-data/gis-data-and-databases). 

This data is then filtered to our desired area of interest, clipped to the extent of the study area, and saved as a GeoPackage file.

Finally, the data is loaded for exploratory data analysis.

#### Import libraries

In [1]:
import requests
import zipfile
import os
import pandas as pd
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

____________________

#### Download data

In [2]:
# Define the URL and save path for the zip file
url = 'https://fortress.wa.gov/dnr/geologydata/publications/data_download/ger_portal_subsurface_database.zip'
save_path = '../data/temp/ger_portal_subsurface_database.zip'
extract_path = '../data/temp/extracted_files/'

In [3]:
# Ensure the target folders exist
os.makedirs(os.path.dirname(save_path), exist_ok=True)
os.makedirs(extract_path, exist_ok=True)

In [4]:
try:
    with requests.get(url, stream=True) as response:
        response.raise_for_status()
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):  # Download in chunks
                file.write(chunk)
    print(f"Downloaded file saved to {save_path}")
except requests.exceptions.RequestException as e:
    print(f"Failed to download the file. Error: {e}")
    exit(1)

Downloaded file saved to ../data/temp/ger_portal_subsurface_database.zip


In [5]:
# Check if the file is a valid ZIP file and extract files
if zipfile.is_zipfile(save_path):
    try:
        with zipfile.ZipFile(save_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
        print(f"Files extracted to {extract_path}")
    except zipfile.BadZipFile as e:
        print(f"Error: The file at {save_path} is a bad ZIP file. {str(e)}")
else:
    print(f"Error: The file at {save_path} is not recognized as a valid ZIP file.")

Files extracted to ../data/temp/extracted_files/


____________________

#### Filter and clip data

In [6]:
# View the extracted files
extracted_files = os.listdir(extract_path)
print(f"Extracted files: {extracted_files}")

Extracted files: ['README_subsurface.docx', 'WGS_Subsurface_Database.gdb', 'metadata', 'Subsurface_Database.mpkx', 'layer_files']


In [7]:
# Open the geodatabase file: 'WGS_Subsurface_Database.gdb' in the extracted files
gdb_path = os.path.join(extract_path, 'WGS_Subsurface_Database.gdb') 

# List all layers in the geodatabase
layers = fiona.listlayers(gdb_path)
print(f"Layers in the geodatabase: {layers}")

Layers in the geodatabase: ['bedrock', 'hydrologic', 'layer_comments', 'documents', 'in_situ_test_data', 'borehole_info', 'layer_data', 'lithology_logs', 'water_wells', 'subsurface_documents']


In [ ]:
# We are interested in the layer_data layer table
layer_data = gpd.read_file(gdb_path, layer='layer_data')
print(f"Shape of the layer_data table: {layer_data.shape}")

print("Columns:", layer_data.columns)

# Display the first few rows of the layer_data table
layer_data.head()

In [ ]:
# Only keep the columns of interest
columns_of_interest = ['BOREHOLE_ID', 'LAYER_NUMBER', 'TOP_DEPTH_FT', 'BOTTOM_DEPTH_FT', 'USCS']

# Filter the layer_data table to only include the columns of interest
layer_data_filtered = layer_data[columns_of_interest]

# Display the first few rows of the filtered table
layer_data_filtered.head()

In [ ]:
# Make a column of "simplified USCS" for our purposes
layer_data_filtered['SIMPLE_USCS'] = layer_data_filtered['USCS'].str[0]

In [ ]:
# Also read the 'borehole_info' table, which contains information about the boreholes such as location, depth, etc.
borehole_info = gpd.read_file(gdb_path, layer='borehole_info')
print(f"Shape of the borehole_info table: {borehole_info.shape}")

print("Columns:", borehole_info.columns)

# Display the first few rows of the borehole_info table
borehole_info.head()

In [ ]:
# Only keep the columns of interest
columns_of_interest = ['BOREHOLE_ID', 'BOREHOLE_NAME', 'BOREHOLE_TYPE', 'BOREHOLE_DEPTH_FT', 'ELEVATION_FT', 'LATITUDE', 'LONGITUDE']

# Filter the borehole_info table to only include the columns of interest
borehole_info_filtered = borehole_info[columns_of_interest]

# Display the first few rows of the filtered table
borehole_info_filtered.head()

In [ ]:
# Combine the two tables based on the 'BOREHOLE_ID' column, note that there are multiple layer_data entries for each borehole
borehole_data = pd.merge(borehole_info_filtered, layer_data_filtered, on='BOREHOLE_ID', how='inner')
print(f"Shape of the merged table: {borehole_data.shape}")

# Display the first few rows of the merged table
borehole_data.head()

In [ ]:
# We are only interested in 'Geotechnical' borehole types
borehole_data = borehole_data[borehole_data['BOREHOLE_TYPE'] == 'Geotechnical']

# Display the first few rows of the filtered table
borehole_data.head()

In [ ]:
# Drop any rows with "Unknown" or "Not Applicable" values in the 'USCS' column
# borehole_data = borehole_data[(borehole_data['USCS'] != 'Unknown') & ('Not Applicable' not in borehole_data['USCS']) & ('Not applicable' not in borehole_data['USCS']) & ('Now applicable' not in borehole_data['USCS']) & ('Not applicabble' not in borehole_data['USCS']) & ('Not application' not in borehole_data['USCS'])]
# print(f"Shape of the filtered table: {borehole_data.shape}")
# Define the regex pattern to match variations of "Unknown" and "Not Applicable"
pattern = r'Unknown|Not\s*Applicable|Not\s*applicable|Now\s*applicable|Not\s*applicabble|Not\s*application|Non-standard'

# Drop rows where the 'USCS' column contains any of the specified patterns
borehole_data = borehole_data[~borehole_data['USCS'].str.contains(pattern, case=False, na=False)]

# Compute the thickness of each layer
borehole_data['LAYER_THICKNESS_FT'] = borehole_data['BOTTOM_DEPTH_FT'] - borehole_data['TOP_DEPTH_FT']

# Display the first few rows of the table with the thickness column
borehole_data.head()

In [ ]:
# Write the final table to a CSV file
output_file = '../data/0-borehole_data.csv'
borehole_data.to_csv(output_file, index=False)

____________________

#### Exploratory Data Analysis

In [ ]:
# In the first iteration of this project, we will only try to predict the USCS and layer thickness of the uppermost soil layer (Layer 1).
# Read in the CSV file
borehole_data = pd.read_csv(output_file)

# Filter the data to only include Layer 1
layer_1_data = borehole_data[borehole_data['LAYER_NUMBER'] == 1]

# Display the first few rows of the filtered table
layer_1_data.head()

In [ ]:
# We are also only interested in a small region around the University of Washington, Seattle, to begin
# Define the bounding box for the region around the University of Washington, Seattle
min_lon, max_lon = -122.35, -122.3
min_lat, max_lat = 47.65, 47.7

# Filter the data to only include the region around the University of Washington, Seattle
uw_layer1_data = layer_1_data[(layer_1_data['LONGITUDE'] >= min_lon) & (layer_1_data['LONGITUDE'] <= max_lon) & (layer_1_data['LATITUDE'] >= min_lat) & (layer_1_data['LATITUDE'] <= max_lat)]

# Display the first few rows of the filtered table
uw_layer1_data

In [ ]:
# Layer 1 statistics
uw_layer1_data.describe()

In [ ]:
# Display the unique values in the 'USCS' column and their counts
uscs_counts = uw_layer1_data['USCS'].value_counts()
print(uscs_counts)

In [ ]:
# Display the unique values in the 'USCS' column and their counts
uscs_counts = uw_layer1_data['SIMPLE_USCS'].value_counts()
print(uscs_counts)

In [ ]:
# Make a histogram of the layer thickness values
plt.figure(figsize=(10, 6))
sns.histplot(uw_layer1_data['LAYER_THICKNESS_FT'], bins=20, kde=True)

In [ ]:
# Save the filtered data to a new CSV file
output_file = '../data/1-uw_layer1_data.csv'
uw_layer1_data.to_csv(output_file, index=False)

____________________

Source: 
1. Washington Geological Survey, 2023, Subsurface database--GIS data, July 2023: Washington Geological Survey Digital Data Series 11, version 2.3, previously released March 2023. https://fortress.wa.gov/dnr/geologydata/publications/data_download/ger_portal_subsurface_database.zip
